In [2]:
import cv2
import random
# import GPyOpt as gy
# import noise as ns
import tensorflow as tf
import torch
import torchvision
from torchvision import transforms
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import io
import os
import math
import keras

# import utility and helper functions/classes 
from src.model import *
from src.dataloader import * 
from src.utils import * 


import time as time


mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

2023-12-07 22:42:17.466244: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 22:42:17.466300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 22:42:17.467716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 22:42:17.476572: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 22:42:18.635595: W tensorflow/compiler/tf2

In [7]:
def getModel(dataset, device): 
    if dataset == "imagenet64": 
        pretrained_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True).to(device) 
        pretrained_model.eval()
        return pretrained_model 
    
    elif dataset == "butterflies_and_moths": 
        pretrained_model = keras.models.load_model(os.path.join("HSJattack", "EfficientNetB0_butterfly.h5"), custom_objects={'F1_score':'F1_score'}) 
        pretrained_model.trainable = False 
        return pretrained_model 
    
    else: 
        raise Exception("Not a valid dataset name.")

In [8]:
torch_model = getModel("imagenet64", "cuda:1")
flow_model = getModel("butterflies_and_moths", "cuda:1")

Using cache found in /hpc/home/mb625/.cache/torch/hub/pytorch_vision_v0.10.0
/scratch/mb625/anaconda3/envs/DL_final/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/mb625/anaconda3/envs/DL_final/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# get a random subset of "dataset_size" validation dataset image filepaths and store them in a list 
image_filepaths = getSubset("imagenet64", 100, True)

# get class2index and index2 class dictionaries for easy querying 
image_cls2idx = class2index("imagenet64") 
image_idx2cls = {v: k for k, v in image_cls2idx.items()}

In [17]:

# test out for pytorch 
for i, imgpath in enumerate(image_filepaths): 
    rawimage = Image.open(imgpath)
    rawimage = tf.keras.preprocessing.image.img_to_array(rawimage) / 255
    rawimage = cv2.resize(rawimage, dsize=(224, 224))
    
    # image = torch.tensor(rawimage, dtype=torch.float32).to("cuda")
    # image = torch.unsqueeze(image, 0)
    # print(image.shape)
    # prediction = predict(image, torch_model, is_torch=True)

    image = tf.cast(rawimage, tf.float32)
    image = image[None, ...]
    print(image.shape, type(image))
    prediction = flow_model.predict(image * 255, steps=1) 
    prediction = torch.tensor(prediction) 
    print(prediction.shape)
    print(predict(image, flow_model, is_torch=False).shape)
    
    
    
    print(prediction.shape) 
    print(type(prediction))        
    
    break 

(1, 224, 224, 3) <class 'tensorflow.python.framework.ops.EagerTensor'>
1/1 [==============================] - 0s 87ms/step
torch.Size([1, 100])
1/1 [==============================] - 0s 84ms/step
torch.Size([1, 100])
torch.Size([1, 100])
<class 'torch.Tensor'>


In [11]:
for i, imgpath in enumerate(image_filepaths): 
    rawimage = np.array(Image.open(imgpath)) 
    rawimage = cv2.resize(rawimage, dsize=(299, 299))
    
    image = torch.tensor(rawimage, dtype=torch.float32).to("cuda")
    if image.size() == (299, 299): 
        # if image is grayscale, then repeat the dimensions 
        image = image.unsqueeze(-1) 
        image = image.repeat(1, 1, 3)
    image = torch.unsqueeze(image, 0)
    
    # pred = predict(image, torch_model, is_torch=True, device="cuda")
    flow_model.predict(image)

TypeError: Cannot convert the argument `type_value`: torch.float32 to a TensorFlow DType.

In [10]:
for i, imgpath in enumerate(image_filepaths): 
    rawimage = np.array(Image.open(imgpath)) / 255
    rawimage = cv2.resize(rawimage, dsize=(299, 299))
    
    image = torch.tensor(rawimage, dtype=torch.float32).to("cuda")
    if image.size() == (299, 299): 
        # if image is grayscale, then repeat the dimensions 
        image = image.unsqueeze(-1) 
        image = image.repeat(1, 1, 3)
    image = torch.unsqueeze(image, 0)
    
    pred = predict(image, torch_model, is_torch=True, device="cuda")
    
    print(pred.shape)

torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1000])
torch.Size

KeyboardInterrupt: 

In [ ]:
def predict(image, model, is_torch=True, device='cuda'):
    """
    input: normalized tensor of shape (B, C, H, W)
    output: numpy array of predictions
    """
    # print("predicting")
    if is_torch: 
        with torch.no_grad():
            preds = model(transforming(image).to(device))
        return preds
    else: 
        pass 